In [ ]:
import os
from collections import defaultdict
import numpy as np
from matplotlib import pyplot as plt
import random
from queue import PriorityQueue

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Bidirectional, RepeatVector, Concatenate, Activation
from tensorflow.keras.activations import softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.optimizers import Adam

In [ ]:
def load_image_list(filename):
    with open(filename,'r') as image_list_f: 
        return [line.strip() for line in image_list_f]    

In [ ]:
train_list = load_image_list(os.path.join(FLICKR_PATH, 'Flickr_8k.trainImages.txt'))
dev_list = load_image_list(os.path.join(FLICKR_PATH,'Flickr_8k.devImages.txt'))
test_list = load_image_list(os.path.join(FLICKR_PATH,'Flickr_8k.testImages.txt'))

In [ ]:
len(train_list), len(dev_list), len(test_list)

In [ ]:
dev_list[20]

In [ ]:
IMG_PATH = os.path.join(FLICKR_PATH, "Flickr8k_Dataset")

In [ ]:
import numpy as np
import PIL
from matplotlib import pyplot as plt

In [ ]:
image = PIL.Image.open(os.path.join(IMG_PATH, dev_list[20]))
image

If cant't see the image, try

In [ ]:
plt.imshow(image)